In [8]:

import sys
from aubio import tempo, source, pitch


#  To autoreload when new libraries are install in os shell
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:

catalog_dir = '/Users/sidabid09/projects/loopcrafthome/catalog'

# based on aubio python examples https://github.com/aubio/aubio/tree/master/python/demos

# tempo identification using aubio

In [7]:


win_s = 512                 # fft size
hop_s = win_s // 2          # hop size

filename = f'{catalog_dir}/looperman-l-0330969-0039591-dfury-bangin-groove.wav'



samplerate = 44100



s = source(filename, samplerate, hop_s)
samplerate = s.samplerate
o = tempo("default", win_s, hop_s, samplerate)

# tempo detection delay, in samples
# default to 4 blocks delay to catch up with
delay = 4. * hop_s

# list of beats, in samples
beats = []

# total number of frames read
total_frames = 0
while True:
    samples, read = s()
    is_beat = o(samples)
    if is_beat:
        this_beat = int(total_frames - delay + is_beat[0] * hop_s)
        print("%f" % (this_beat / float(samplerate)))
        beats.append(this_beat)
    total_frames += read
    if read < hop_s: break
#print len(beats)

1.632744
1.864921
2.097075
2.329252
2.561429
2.793605
3.407234
3.877143
4.347052
4.731043
5.202834
5.674626
6.149297
6.620748
7.092200


# key detection using aubio


In [10]:




filename = f'{catalog_dir}/looperman-l-0330969-0039591-dfury-bangin-groove.wav'

downsample = 1
samplerate = 44100 // downsample

win_s = 4096 // downsample # fft size
hop_s = 512  // downsample # hop size

s = source(filename, samplerate, hop_s)
samplerate = s.samplerate

tolerance = 0.8

pitch_o = pitch("yin", win_s, hop_s, samplerate)
pitch_o.set_unit("midi")
pitch_o.set_tolerance(tolerance)

pitches = []
confidences = []

# total number of frames read
total_frames = 0
while True:
    samples, read = s()
    pitch = pitch_o(samples)[0]
    #pitch = int(round(pitch))
    confidence = pitch_o.get_confidence()
    #if confidence < 0.8: pitch = 0.
    print("%f %f %f" % (total_frames / float(samplerate), pitch, confidence))
    pitches += [pitch]
    confidences += [confidence]
    total_frames += read
    if read < hop_s: break

if 0: sys.exit(0)

#print pitches
import os.path
from numpy import array, ma
import matplotlib.pyplot as plt
from demo_waveform_plot import get_waveform_plot, set_xlabels_sample2time

skip = 1

pitches = array(pitches[skip:])
confidences = array(confidences[skip:])
times = [t * hop_s for t in range(len(pitches))]

fig = plt.figure()

ax1 = fig.add_subplot(311)
ax1 = get_waveform_plot(filename, samplerate = samplerate, block_size = hop_s, ax = ax1)
plt.setp(ax1.get_xticklabels(), visible = False)
ax1.set_xlabel('')

def array_from_text_file(filename, dtype = 'float'):
    filename = os.path.join(os.path.dirname(__file__), filename)
    return array([line.split() for line in open(filename).readlines()],
        dtype = dtype)

ax2 = fig.add_subplot(312, sharex = ax1)
ground_truth = os.path.splitext(filename)[0] + '.f0.Corrected'
if os.path.isfile(ground_truth):
    ground_truth = array_from_text_file(ground_truth)
    true_freqs = ground_truth[:,2]
    true_freqs = ma.masked_where(true_freqs < 2, true_freqs)
    true_times = float(samplerate) * ground_truth[:,0]
    ax2.plot(true_times, true_freqs, 'r')
    ax2.axis( ymin = 0.9 * true_freqs.min(), ymax = 1.1 * true_freqs.max() )
# plot raw pitches
ax2.plot(times, pitches, '.g')
# plot cleaned up pitches
cleaned_pitches = pitches
#cleaned_pitches = ma.masked_where(cleaned_pitches < 0, cleaned_pitches)
#cleaned_pitches = ma.masked_where(cleaned_pitches > 120, cleaned_pitches)
cleaned_pitches = ma.masked_where(confidences < tolerance, cleaned_pitches)
ax2.plot(times, cleaned_pitches, '.-')
#ax2.axis( ymin = 0.9 * cleaned_pitches.min(), ymax = 1.1 * cleaned_pitches.max() )
#ax2.axis( ymin = 55, ymax = 70 )
plt.setp(ax2.get_xticklabels(), visible = False)
ax2.set_ylabel('f0 (midi)')

# plot confidence
ax3 = fig.add_subplot(313, sharex = ax1)
# plot the confidence
ax3.plot(times, confidences)
# draw a line at tolerance
ax3.plot(times, [tolerance]*len(confidences))
ax3.axis( xmin = times[0], xmax = times[-1])
ax3.set_ylabel('confidence')
set_xlabels_sample2time(ax3, times[-1], samplerate)
plt.show()
#plt.savefig(os.path.basename(filename) + '.svg')


0.000000 0.000000 0.000000
0.011610 0.000000 0.000000
0.023220 0.000000 0.000000
0.034830 0.000000 0.000000
0.046440 48.967514 0.473591
0.058050 46.961128 0.447872
0.069660 46.023552 0.496152
0.081270 45.777657 0.636971
0.092880 44.412525 0.707891
0.104490 44.140873 0.546786
0.116100 44.630394 0.367006
0.127710 44.826149 0.250985
0.139320 41.683201 0.347022
0.150930 44.967587 0.276501
0.162540 43.761513 0.304388
0.174150 44.622398 0.366995
0.185760 44.255108 0.348583
0.197370 44.230450 0.317754
0.208980 44.953648 0.282064
0.220590 43.083149 0.314088
0.232200 43.594574 0.263089
0.243810 43.287521 0.331789
0.255420 43.526287 0.295289
0.267029 43.490883 0.244167
0.278639 31.137024 0.947336
0.290249 31.190132 0.913393
0.301859 31.151035 0.897602
0.313469 31.184376 0.889643
0.325079 31.179581 0.897236
0.336689 31.176224 0.912417
0.348299 31.185455 0.926046
0.359909 31.175312 0.937912
0.371519 31.170643 0.946251
0.383129 31.173687 0.947897
0.394739 31.174114 0.948550
0.406349 31.165035 0.949

ModuleNotFoundError: No module named 'matplotlib'